# 1. Librerias

In [ ]:
import os
from pathlib import Path
from datetime import datetime
import pandas as pd
import glob
import numpy as np

# 2. Rutas

In [ ]:
# ============================================================================
# ⚙️ PARÁMETROS DE CONFIGURACIÓN MANUAL
# ============================================================================
# 🔧 ACTUALIZA ESTOS VALORES CADA MES:

# 1. ESTRUCTURA DE CARPETAS (Año/Mes/Día)
CARPETA_ANO = "2026"
CARPETA_MES = "02_Febrero"
CARPETA_DIA = "10"

# 2. ARCHIVOS SIE (con fecha YYYYMMDD en el nombre)
FECHA_ARCHIVO = "10022026"  # Formato: DDMMYYYY
ARCHIVO_S1_AUTOMATICO = f"AUTOMATICOS-S1EPS025{FECHA_ARCHIVO}.VAL"
ARCHIVO_S1_VAL = f"S1EPS025{FECHA_ARCHIVO}.VAL"
ARCHIVO_S3 = f"S3EPS025{FECHA_ARCHIVO}.TXT"

# 3. ARCHIVOS DE VALIDACIÓN
FECHA_REPORTE = "2026_02_12"  # Formato: YYYY_MM_DD
ARCHIVO_MS_SIE = f"Reporte_Validación Archivos Maestro_{FECHA_REPORTE}.csv"
ARCHIVO_RELACIONES_LABORALES = f"Reporte_Afiliados Contributivo Relaciones Laborales_{FECHA_REPORTE}.csv"

# 4. ARCHIVO DE SALIDA
NOMBRE_EXCEL = "Yesid-12-02-2026.xlsx"

# ============================================================================
# DETECCIÓN AUTOMÁTICA DE RUTAS (UNIVERSAL)
# ============================================================================

import os
from datetime import datetime

# Obtener usuario del sistema
usuario = os.environ.get('USERNAME') or os.environ.get('USER')
user_home = os.path.expanduser("~")  # C:\Users\{usuario}

# Rutas base (estructura estándar para todos los usuarios)
ONEDRIVE_BASE = os.path.join(user_home, "OneDrive - 891856000_CAPRESOCA E P S")
ESCRITORIO_BASE = os.path.join(ONEDRIVE_BASE, "Escritorio", "Yesid Rincón Z", "Traslados", "Procesos BDUA")
CAPRESOCA_BASE = os.path.join(ONEDRIVE_BASE, "Capresoca", "AlmostClear", "SIE", "Aseguramiento")

# ============================================================================
# CONSTRUCCIÓN DE RUTAS AUTOMÁTICAS
# ============================================================================

# Rutas de archivos a validar para cargar a SIE
R_CARPETA_SIE = os.path.join(ESCRITORIO_BASE, CARPETA_ANO, CARPETA_MES, CARPETA_DIA, "SIE")
R_S1_Automatico = os.path.join(R_CARPETA_SIE, ARCHIVO_S1_AUTOMATICO)
R_S1_Val = os.path.join(R_CARPETA_SIE, ARCHIVO_S1_VAL)
R_S3 = os.path.join(R_CARPETA_SIE, ARCHIVO_S3)

# Rutas de archivos de validación
R_MS_SIE = os.path.join(CAPRESOCA_BASE, "ms_sie", ARCHIVO_MS_SIE)
R_Expedientes_SIE = os.path.join(CAPRESOCA_BASE, "Expedientes", "Años")
R_Relaciones_Laborales = os.path.join(CAPRESOCA_BASE, "relaciones laborales", ARCHIVO_RELACIONES_LABORALES)

# Ruta de archivo de salida
R_Salida_Excel = os.path.join(R_CARPETA_SIE, NOMBRE_EXCEL)

# ============================================================================
# VALIDACIÓN Y RESUMEN
# ============================================================================

print("=" * 80)
print(f"👤 USUARIO: {usuario}")
print(f"📁 OneDrive: {ONEDRIVE_BASE}")
print("=" * 80)

print(f"\n📋 PARÁMETROS CONFIGURADOS:")
print(f"   📅 Carpeta fecha: {CARPETA_ANO}/{CARPETA_MES}/{CARPETA_DIA}")
print(f"   📄 S1 Automático: {ARCHIVO_S1_AUTOMATICO}")
print(f"   📄 S1 Val: {ARCHIVO_S1_VAL}")
print(f"   📄 S3: {ARCHIVO_S3}")
print(f"   📊 MS SIE: {ARCHIVO_MS_SIE}")
print(f"   📊 Relaciones Laborales: {ARCHIVO_RELACIONES_LABORALES}")
print(f"   💾 Excel salida: {NOMBRE_EXCEL}")

print("\n" + "-" * 80)
print("🔍 VALIDACIÓN DE RUTAS:")
print("-" * 80)

rutas_validar = {
    "📂 Carpeta SIE": R_CARPETA_SIE,
    "📄 S1 Automático": R_S1_Automatico,
    "📄 S1 Val": R_S1_Val,
    "📄 S3": R_S3,
    "📊 MS SIE": R_MS_SIE,
    "📂 Expedientes SIE": R_Expedientes_SIE,
    "📊 Relaciones Laborales": R_Relaciones_Laborales
}

errores = []
for nombre, ruta in rutas_validar.items():
    existe = os.path.exists(ruta)
    print(f"{'✅' if existe else '❌'} {nombre}")
    print(f"   {ruta}")
    if not existe:
        errores.append(nombre)

if errores:
    print(f"\n⚠️ {len(errores)} ruta(s) no encontrada(s)")
    print("\n💡 Verifica:")
    print("   1. Que OneDrive esté sincronizado")
    print("   2. Que los parámetros al inicio estén actualizados")
    print("   3. Que los archivos existan en las carpetas")
    print("   4. Que las fechas en los nombres de archivo sean correctas")
else:
    print(f"\n✅ Todas las rutas validadas correctamente")

print("=" * 80 + "\n")

# 3. Cargue de Archivos

## 3.1. Dataframes

In [ ]:
# Cargar los archivos a validar para subir al SIE
df_S1_Automatico = pd.read_csv(R_S1_Automatico, delimiter=',', dtype=str, encoding='ansi', header=None)
df_S1_Val = pd.read_csv(R_S1_Val, delimiter=',', dtype=str, encoding='ansi', header=None)
df_S3 = pd.read_csv(R_S3, delimiter=',', dtype=str, encoding='ansi', header=None)

# Cargar archivos que sirven para validación
df_MS_SIE = pd.read_csv(R_MS_SIE, delimiter=';', dtype=str, encoding='ansi')
df_Relaciones_Laborales = pd.read_csv(R_Relaciones_Laborales, delimiter=';', dtype=str, encoding='ansi')

In [ ]:
# Listar todos los archivos .TXT en la carpeta R_Expedientes_SIE
txt_files = list(Path(R_Expedientes_SIE).glob("*.TXT"))

# Cargar cada archivo en un dataframe y luego concatenarlos
df_list = [pd.read_csv(file, sep="|", encoding="ANSI", dtype=str) for file in txt_files]

# Concatenar todos los dataframes en uno solo
df_expedientes_SIE = pd.concat(df_list, ignore_index=True)

# 3.2. variables

In [ ]:
filename = os.path.basename(R_S1_Automatico)
const_part = "AUTOMATICOS-S1EPS025"
date_str = filename.replace(const_part, "")[:8]
formatted_date = date_str[:2] + "/" + date_str[2:4] + "/" + date_str[4:]
print(formatted_date)

# 4. Limpiar bases de datos

In [ ]:
print(df_expedientes_SIE.shape[0])
# Filtrar el dataframe para eliminar registros donde "Estado Expediente" sea "Cerrado", "Pendiente" o "Anulado"
df_expedientes_SIE = df_expedientes_SIE[~df_expedientes_SIE["Estado Expediente"].isin(["Pendiente", "Anulado"])]
procesos_validos = [
    "Ingreso Afiliado Contributivo Cotizante",
    "Afiliación por Adición Relación Laboral",
    "Ingreso Afiliado Contributivo Nacimiento",
    "Ingreso Afiliado Contributivo Beneficiario",
    "Actualización Beneficiario ó Adicional a Cotizante",
    "Actualización Relación Laboral"
]

df_expedientes_SIE = df_expedientes_SIE[df_expedientes_SIE["Proceso"].isin(procesos_validos)]
print(df_expedientes_SIE.shape[0])

In [ ]:
# Convertir la columna 'Fecha Grabado' a datetime usando el formato "yyyy/mm/dd hh:mm"
df_expedientes_SIE['Fecha Grabado_dt'] = pd.to_datetime(df_expedientes_SIE['Fecha Grabado'], format='%Y/%m/%d %H:%M')

# Ordenar el dataframe de forma descendente según 'Fecha Grabado_dt'
df_expedientes_SIE = df_expedientes_SIE.sort_values('Fecha Grabado_dt', ascending=False)

# Eliminar duplicados en base a 'Tipo Documento' y 'Número Identificación', conservando el registro más reciente
df_expedientes_SIE = df_expedientes_SIE.drop_duplicates(subset=['Tipo Documento', 'Número Identificación'], keep='first')

# Convertir la fecha al formato "dd/mm/yyyy"
df_expedientes_SIE['Fecha Grabado'] = df_expedientes_SIE['Fecha Grabado_dt'].dt.strftime('%d/%m/%Y')

# Eliminar la columna temporal
df_expedientes_SIE = df_expedientes_SIE.drop(columns=['Fecha Grabado_dt'])

In [ ]:
# Imprimir número de registros antes del proceso
print("Número de registros antes de eliminar duplicados:", df_Relaciones_Laborales.shape[0])

# Convertir la columna "fecha_ingreso" a datetime, asumiendo formato "yyyy-mm-dd"
df_Relaciones_Laborales["fecha_ingreso_dt"] = pd.to_datetime(df_Relaciones_Laborales["fecha_ingreso"], format="%Y-%m-%d")

# Ordenar por la fecha en forma descendente para que el más reciente quede primero
df_Relaciones_Laborales = df_Relaciones_Laborales.sort_values("fecha_ingreso_dt", ascending=False)

# Eliminar duplicados manteniendo solo el registro con la fecha más reciente 
df_Relaciones_Laborales = df_Relaciones_Laborales.drop_duplicates(subset=["tipo_documento", "numero_identificacion"], keep="first")

# Convertir la fecha al formato "dd/mm/yyyy"
df_Relaciones_Laborales["fecha_ingreso"] = df_Relaciones_Laborales["fecha_ingreso_dt"].dt.strftime("%d/%m/%Y")

# Eliminar la columna temporal usada para el manejo de fechas
df_Relaciones_Laborales = df_Relaciones_Laborales.drop(columns=["fecha_ingreso_dt"])

# Imprimir número de registros después del proceso
print("Número de registros después de eliminar duplicados:", df_Relaciones_Laborales.shape[0])

In [ ]:
df_expedientes_SIE.columns

# 5. Validaciones

## 5.1 Validacion S1 automatico

In [ ]:
# Primero creamos dos columnas auxiliares en df_S1_Automatico para facilitar el cruce.
df_S1_Automatico['tipo_documento'] = df_S1_Automatico.iloc[:, 1]
df_S1_Automatico['numero_identificacion'] = df_S1_Automatico.iloc[:, 2]

# Realizamos un merge izquierdo para traer la columna "fecha_ingreso" de df_Relaciones_Laborales
df_S1_Automatico = df_S1_Automatico.merge(
    df_Relaciones_Laborales[['tipo_documento', 'numero_identificacion', 'fecha_ingreso']],
    on=['tipo_documento', 'numero_identificacion'],
    how='left'
)

# Creamos la columna "Relacion Laboral":
# Si 'fecha_ingreso' tiene dato, asignamos "SI", de lo contrario "No"
df_S1_Automatico['Relacion Laboral'] = df_S1_Automatico['fecha_ingreso'].apply(lambda x: 'SI' if pd.notnull(x) else 'No')

# Extraemos las columnas clave y las que queremos agregar del df_expedientes_SIE
df_temp = df_expedientes_SIE[['Tipo Documento', 'Número Identificación', 'Proceso', 'Fecha Grabado', 'Usuario Grabado']]

# Hacemos merge para incorporar 'Proceso' y 'Fecha Grabado' en df_S1_Automatico usando los identificadores
df_S1_Automatico = df_S1_Automatico.merge(
    df_temp,
    left_on=['tipo_documento', 'numero_identificacion'],
    right_on=['Tipo Documento', 'Número Identificación'],
    how='left'
)

# Opcional: eliminamos las columnas duplicadas de llave del merge si no son necesarias
df_S1_Automatico.drop(columns=['Tipo Documento', 'Número Identificación'], inplace=True)

# Usar la columna 'Fecha Grabado' si existe; si no, se utiliza 'fecha_ingreso'
fechas = df_S1_Automatico['Fecha Grabado'].fillna(df_S1_Automatico['fecha_ingreso'])
# Convertir las fechas a datetime conservando el formato dd/mm/yyyy
fecha_registro = pd.to_datetime(fechas, format='%d/%m/%Y', errors='coerce')
# Referencia desde la columna en índice 20
ref_col = pd.to_datetime(df_S1_Automatico.iloc[:, 20], format='%d/%m/%Y', errors='coerce')
# Referencia desde la variable formatted_date
ref_var = pd.to_datetime(formatted_date, format='%d/%m/%Y', errors='coerce')

# Crear la nueva columna: si la fecha registrada es mayor o igual a cualquiera de las referencias, se asigna "R1 no SIE"
df_S1_Automatico['Nueva Columna'] = np.where((fecha_registro >= ref_col) | (fecha_registro >= ref_var),
                                              "R1 no SIE", "S1 SIE")
df_S1_Automatico.loc[df_S1_Automatico['fecha_ingreso'].isnull() | (df_S1_Automatico['fecha_ingreso'] == ''), 'Nueva Columna'] = "S1 SIE"

In [ ]:
# Filtrar los registros que cumplen las condiciones adicionales
condiciones_adicionales = (df_S1_Automatico['Usuario Grabado'].notnull()) & \
                          (df_S1_Automatico['Usuario Grabado'] != '') & \
                          (df_S1_Automatico['Usuario Grabado'] != 'Jhonatan.perez')

# Máscara original de fechas y condiciones adicionales
mask_fechas = (fecha_registro >= ref_col) | (fecha_registro >= ref_var)
mask_adicionales = condiciones_adicionales

# Nueva máscara: fecha_ingreso no nula ni vacía
mask_fecha_ingreso = df_S1_Automatico['fecha_ingreso'].notna() & (df_S1_Automatico['fecha_ingreso'] != '')

# Combinas todo
mask_r1 = mask_fechas & mask_adicionales & mask_fecha_ingreso

# Asignas la nueva columna
df_S1_Automatico['Nueva Columna'] = np.where(
    mask_r1,
    "R1 no SIE",
    "S1 SIE"
)

## 5.2. R1

In [ ]:
Df_R1 = df_S1_Automatico[
    (df_S1_Automatico["Nueva Columna"] == "R1 no SIE") & (df_S1_Automatico.iloc[:, 30] == "F")
][["tipo_documento", "numero_identificacion"]]

# Realizamos un merge izquierdo para traer la columna "fecha_ingreso" de df_Relaciones_Laborales
Df_R1 = Df_R1.merge(
    df_Relaciones_Laborales[['tipo_documento', 'numero_identificacion', 'tipo_documento_aportante', 'numero_identificacion_aportante', 'fecha_ingreso']],
    on=['tipo_documento', 'numero_identificacion'],
    how='left'
)

# 6. Guardar Datos

In [ ]:
with pd.ExcelWriter(R_Salida_Excel) as writer:
    df_S1_Automatico.to_excel(writer, index=False, sheet_name='S1Automatico')
    Df_R1.to_excel(writer, index=False, sheet_name='Df_R1')
    df_S1_Val.to_excel(writer, index=False, sheet_name='df_S1_Val')
    df_S3.to_excel(writer, index=False, sheet_name='df_S3')